In [ ]:
!pip install opendatasets
import opendatasets as od

In [ ]:
od.download('https://www.kaggle.com/datasets/maheshmitta12/ecg-dataset')

{"username":"maheshmitta12","key":"26616f230e68dc3dba1537b000a70bcc"}

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: maheshmitta12
Your Kaggle Key: ··········


100%|██████████| 788M/788M [00:13<00:00, 61.6MB/s]


{'username': 'maheshmitta12', 'key': '26616f230e68dc3dba1537b000a70bcc'}

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing import image
import numpy as np


In [ ]:
# Set your data directory
data_directory = '/content/ecg-dataset/dataset/ECG_Image_data/train'

In [ ]:
# Define image size and batch size
img_size = (128, 128)
batch_size = 32

In [ ]:
# Create data generators for training, validation, and testing
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    data_directory,
    target_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale',  # Use grayscale images
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_directory,
    target_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

test_generator = datagen.flow_from_directory(
    data_directory,
    target_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False  # Don't shuffle for evaluation
)

Found 72638 images belonging to 5 classes.
Found 18156 images belonging to 5 classes.
Found 90794 images belonging to 5 classes.


In [ ]:
# Create the LeNet-5 model
model = models.Sequential([
    layers.Conv2D(6, (5, 5), activation='relu', input_shape=(128, 128, 1)),  # Use 'grayscale' images, so input_shape=(128, 128, 1)
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (5, 5), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(120, activation='relu'),
    layers.Dense(84, activation='relu'),
    layers.Dense(5, activation='softmax')  # 5 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model

epochs = 10
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/10
2270/2270 [==============================] - 1074s 473ms/step - loss: 0.0502 - accuracy: 0.9849 - val_loss: 0.0230 - val_accuracy: 0.9939
Epoch 2/10
2270/2270 [==============================] - 1052s 463ms/step - loss: 0.0137 - accuracy: 0.9956 - val_loss: 0.0111 - val_accuracy: 0.9969
Epoch 3/10
2270/2270 [==============================] - 1048s 462ms/step - loss: 0.0201 - accuracy: 0.9943 - val_loss: 0.0091 - val_accuracy: 0.9975
Epoch 4/10
2270/2270 [==============================] - 996s 439ms/step - loss: 0.0133 - accuracy: 0.9959 - val_loss: 0.0093 - val_accuracy: 0.9974
Epoch 5/10
2270/2270 [==============================] - 1051s 463ms/step - loss: 0.0119 - accuracy: 0.9965 - val_loss: 0.0085 - val_accuracy: 0.9976
Epoch 6/10
2270/2270 [==============================] - 1042s 459ms/step - loss: 0.0050 - accuracy: 0.9983 - val_loss: 0.0081 - val_accuracy: 0.9979
Epoch 7/10
2270/2270 [==============================] - 1040s 458ms/step - loss: 0.0289 - accuracy: 0.9942 

In [ ]:
model.save('LeNet_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Evaluate the model on the test set
test_results = model.evaluate(test_generator)

# Print test metrics
print("Test Loss:", test_results[0])
print("Test Accuracy:", test_results[1])

2838/2838 [==============================] - 547s 193ms/step - loss: 0.0033 - accuracy: 0.9992
Test Loss: 0.0032715369015932083
Test Accuracy: 0.9992400407791138


In [ ]:
# Generate predictions on the test set
predictions = model.predict(test_generator)

2838/2838 [==============================] - 485s 171ms/step


In [ ]:
# Convert predictions to class labels
predicted_classes = tf.argmax(predictions, axis=1)
true_classes = test_generator.classes

In [ ]:
# Print classification report and confusion matrix
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print(conf_matrix)


Classification Report:
              precision    recall  f1-score   support

           F       1.00      1.00      1.00       642
           N       1.00      1.00      1.00     75709
           Q       1.00      1.00      1.00      6431
           S       0.99      0.98      0.99      2223
           V       0.99      1.00      0.99      5789

    accuracy                           1.00     90794
   macro avg       1.00      0.99      1.00     90794
weighted avg       1.00      1.00      1.00     90794


Confusion Matrix:
[[  640     0     0     0     2]
 [    0 75706     3     0     0]
 [    0     2  6429     0     0]
 [    0     0     0  2176    47]
 [    0     0     0    15  5774]]


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/LeNet_model.h5')

In [7]:
# Test an individual image
from tensorflow.keras.preprocessing import image
import numpy as np

class_labels = ['F', 'N', 'Q', 'S', 'V']

def test_single_image(image_path):
    img = image.load_img(image_path, target_size=(128, 128), color_mode='grayscale')
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize pixel values
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    predicted_label = class_labels[predicted_class]
    print(f"Predicted class: {predicted_label}")


# Example usage for testing a single image
test_single_image('/content/N54.png')

1/1 [==============================] - 0s 46ms/step
Predicted class: N
